# <a id='model-selection-and-evaluation'></a> 3. [**Sélection et évaluation de modèle**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#model-selection-and-evaluation)</br>([*Model selection and evaluation*](https://scikit-learn.org/stable/model_selection.html#model-selection-and-evaluation))

# 3.2. [**Réglage des hyper-paramètres d'un estimateur**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#tuning-the-hyper-parameters-of-an-estimator)<br/>([_Tuning the hyper-parameters of an estimator_](https://scikit-learn.org/stable/model_selection.html#tuning-the-hyper-parameters-of-an-estimator))

# Sommaire

- **Volume** : 18 pages, 9 exemples, 3 papiers
- 3.2.1. [**Recherche exhaustive en grille**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#exhaustive-grid-search)<br/>([_Exhaustive Grid Search_](https://scikit-learn.org/stable/model_selection.html#exhaustive-grid-search))
- 3.2.2. [**Optimisation aléatoire des paramètres**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#randomized-parameter-optimization)<br/>([_Randomized Parameter Optimization_](https://scikit-learn.org/stable/model_selection.html#randomized-parameter-optimization))
- 3.2.3. [**Recherche de paramètres optimaux par halving successifs**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#searching-for-optimal-parameters-with-successive-halving)<br/>([_Searching for optimal parameters with successive halving_](https://scikit-learn.org/stable/model_selection.html#searching-for-optimal-parameters-with-successive-halving))
- 3.2.4. [**Conseils pour la recherche de paramètres**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#tips-for-parameter-search)<br/>([_Tips for parameter search_](https://scikit-learn.org/stable/model_selection.html#tips-for-parameter-search))
- 3.2.5. [**Alternatives à la recherche de paramètres par force brute**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/3_model_selection_and_evaluation.ipynb#alternatives-to-brute-force-parameter-search)<br/>([_Alternatives to brute force parameter search_](https://scikit-learn.org/stable/model_selection.html#alternatives-to-brute-force-parameter-search))

Les hyperparamètres sont des paramètres qui ne sont pas directement appris au sein des estimateurs. Dans scikit-learn, ils sont transmis en tant qu'arguments au constructeur des classes d'estimateur. Des exemples typiques incluent `C`, `kernel` et `gamma` pour le classifieur à vecteurs de support, `alpha` pour Lasso, etc.

Il est possible et recommandé d'explorer l'espace des hyperparamètres pour obtenir le meilleur score de [**validation croisée** (3.1)](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation).

Tout paramètre fourni lors de la construction d'un estimateur peut être optimisé de cette manière. Pour trouver les noms et les valeurs actuelles de tous les paramètres pour un estimateur donné, utilisez :

```python
estimator.get_params()
```

Une recherche comprend :
- un estimateur (régresseur ou classifieur tel que `sklearn.svm.SVC()`);
- un espace de paramètres ;
- une méthode pour rechercher ou échantillonner des candidats ;
- un schéma de validation croisée ; et
- une [**fonction de score** (3.2.4.1)](https://scikit-learn.org/stable/modules/grid_search.html#gridsearch-scoring).

Deux approches génériques de recherche de paramètres sont fournies dans scikit-learn : pour des valeurs données, [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) considère exhaustivement toutes les combinaisons de paramètres, tandis que [**`RandomizedSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) peut échantillonner un nombre donné de candidats dans un espace de paramètres avec une distribution spécifiée. Ces deux outils ont des contreparties successives, [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) et [**`HalvingRandomSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html), qui peuvent être beaucoup plus rapides pour trouver une bonne combinaison de paramètres.

Après avoir décrit ces outils, nous détaillons les [**meilleures pratiques** (3.2.4)](https://scikit-learn.org/stable/modules/grid_search.html#grid-search-tips) applicables à ces approches. Certains modèles permettent des stratégies de recherche de paramètres spécialisées et efficaces, décrites dans [**Alternatives to brute force parameter search** (3.2.5)](https://scikit-learn.org/stable/modules/grid_search.html#alternative-cv).

Notez qu'il est courant qu'un petit sous-ensemble de ces paramètres puisse avoir un impact important sur les performances prédictives ou calculatoires du modèle, tandis que d'autres peuvent être laissés à leurs valeurs par défaut. Il est recommandé de lire la documentation de la classe d'estimateur pour mieux comprendre leur comportement attendu, éventuellement en lisant la référence contenue dans la documentation.

## <a id='exhaustive-grid-search'></a> 3.2.1. Recherche exhaustive en grille

La recherche en grille fournie par [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) génère de manière exhaustive des candidats à partir d'une grille de valeurs de paramètres spécifiée avec le paramètre `param_grid`. Par exemple, la grille suivante :

In [1]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
]

L'instance [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) implémente l'API habituelle de l'estimateur : lorsqu'elle est "ajustée" sur un ensemble de données, toutes les combinaisons possibles de valeurs de paramètres sont évaluées et la meilleure combinaison est conservée.

### Exemples

#### [**Stratégie personnalisée de réajustement d'une recherche en grille avec validation croisée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_grid_search_digits.ipynb)<br/>([_Custom refit strategy of a grid search with cross-validation_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_digits.html))

Un exemple de calcul de recherche en grille sur l'ensemble de données "digits".

#### [**Exemple de pipeline pour l'extraction et l'évaluation de caractéristiques de texte**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_grid_search_text_feature_extraction.ipynb)<br/>([_Sample pipeline for text feature extraction and evaluation_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_text_feature_extraction.html))
Un exemple d'utilisation de la recherche en grille couplant des paramètres d'un extracteur de caractéristiques de documents texte (comptage des n-grammes et transformation TF-IDF) avec un classifieur (ici un SVM linéaire entraîné avec SGD avec soit une régularisation de type elastic net, soit une régularisation de type L2) à l'aide d'une instance `pipeline.Pipeline`.

#### [**Validation croisée imbriquée vs. non-imbriquée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_nested_cross_validation_iris.ipynb)<br/>([_Nested versus non-nested cross-validation_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html))

Un exemple de recherche en grille dans une boucle de validation croisée sur l'ensemble de données "Iris". Il s'agit de la meilleure pratique pour évaluer les performances d'un modèle avec la recherche en grille.

#### [**Démonstration de l'évaluation multi-métrique sur `cross_val_score` et `GridSearchCV`**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_multi_metric_evaluation.ipynb)<br/>([_Demonstration of multi-metric evaluation on `cross_val_score` and `GridSearchCV`_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_multi_metric_evaluation.html))

Un exemple d'utilisation de [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) pour évaluer simultanément plusieurs métriques.

#### [**Équilibrer la complexité du modèle et le score validé par validation croisée**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_grid_search_refit_callable.ipynb)<br/>([_Balance model complexity and cross-validated score_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_refit_callable.html))

Un exemple d'utilisation de l'interface `refit=callable` dans [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). L'exemple montre comment cette interface ajoute une certaine flexibilité pour identifier le meilleur estimateur. Cette interface peut également être utilisée dans l'évaluation de plusieurs métriques.

#### [**Comparaison statistique des modèles à l'aide de la recherche en grille**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_grid_search_stats.ipynb)<br/>([_Statistical comparison of models using grid search_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_grid_search_stats.html))

Un exemple de comparaison statistique des résultats de [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html).

## <a id='randomized-parameter-optimization'></a> 3.2.2. Optimisation de paramètres aléatoire

Bien que l'utilisation d'une grille de paramètres soit actuellement la méthode la plus largement utilisée pour l'optimisation des paramètres, d'autres méthodes de recherche présentent des propriétés plus favorables. [**`RandomizedSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) met en œuvre une recherche aléatoire des paramètres, où chaque paramètre est échantillonné à partir d'une distribution de valeurs de paramètres possibles. Cela présente deux avantages principaux par rapport à une recherche exhaustive :

- Un budget peut être choisi indépendamment du nombre de paramètres et des valeurs possibles.
- L'ajout de paramètres qui n'influencent pas les performances n'affecte pas l'efficacité.

La spécification de la manière dont les paramètres doivent être échantillonnés est effectuée à l'aide d'un dictionnaire, très similaire à la spécification des paramètres pour [**`GridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). De plus, un budget de calcul, représenté par le nombre de candidats échantillonnés ou d'itérations d'échantillonnage, est spécifié à l'aide du paramètre `n_iter`. Pour chaque paramètre, il est possible de spécifier soit une distribution de valeurs possibles, soit une liste de choix discrets (qui seront échantillonnés uniformément) :

In [ ]:
import scipy
{
    'C': scipy.stats.expon(scale=100),
    'gamma': scipy.stats.expon(scale=.1),
    'kernel': ['rbf'],
    'class_weight':['balanced', None]
}

Cet exemple utilise le module `scipy.stats`, qui contient de nombreuses distributions utiles pour l'échantillonnage des paramètres, telles que `expon`, `gamma`, `uniform`, `loguniform` ou `randint`.

En principe, n'importe quelle fonction peut être utilisée, pourvu qu'elle fournisse une méthode `rvs` (échantillonnage de variable aléatoire) pour échantillonner une valeur. Un appel à la fonction `rvs` doit fournir des échantillons aléatoires indépendants à partir des valeurs de paramètres possibles lors des appels consécutifs.

> **Attention:** Les distributions dans `scipy.stats` avant la version scipy 0.16 ne permettent pas de spécifier un état aléatoire. À la place, elles utilisent l'état aléatoire global de numpy, qui peut être initialisé via `np.random.seed` ou défini à l'aide de `np.random.set_state`. Cependant, à partir de scikit-learn 0.18, le module [**`sklearn.model_selection`**](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection) définira l'état aléatoire fourni par l'utilisateur si scipy >= 0.16 est également disponible.

Pour les paramètres continus, tels que `C` ci-dessus, il est important de spécifier une distribution continue pour tirer pleinement parti de la randomisation. De cette façon, l'augmentation de `n_iter` conduira toujours à une recherche plus fine.

Une variable aléatoire continue uniforme logarithmique est la version continue d'un paramètre réparti logarithmiquement. Par exemple, pour spécifier l'équivalent de `C` ci-dessus, on peut utiliser `loguniform(1, 100)` au lieu de `[1, 10, 100]`.

En suivant l'exemple ci-dessus de recherche en grille, nous pouvons spécifier une variable aléatoire continue répartie de manière logarithmique entre `1e0` et `1e3` :

In [ ]:
from sklearn.utils.fixes import loguniform
{
    'C': loguniform(1e0, 1e3),
    'gamma': loguniform(1e-4, 1e-3),
    'kernel': ['rbf'],
    'class_weight':['balanced', None]
}

### Exemples

#### [**Comparaison de la recherche aléatoire et de la recherche en grille pour l'estimation d'hyperparamètres**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_randomized_search.ipynb)<br/>([_Comparing randomized search and grid search for hyperparameter estimation_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html))

Compare l'utilisation et l'efficacité de la recherche aléatoire et de la recherche en grille.

### Références

🔬 Bergstra, J. and Bengio, Y., [**“Random search for hyper-parameter optimization”**](https://www.jmlr.org/papers/volume13/bergstra12a/bergstra12a.pdf), The Journal of Machine Learning Research (2012)

## <a id='searching-for-optimal-parameters-with-successive-halving'></a> 3.2.3. Searching for optimal parameters with successive halving

Scikit-learn also provides the [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) and [**`HalvingRandomSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html) estimators that can be used to search a parameter space using successive halving [1] [2]. Successive halving (SH) is like a tournament among candidate parameter combinations. SH is an iterative selection process where all candidates (the parameter combinations) are evaluated with a small amount of resources at the first iteration. Only some of these candidates are selected for the next iteration, which will be allocated more resources. For parameter tuning, the resource is typically the number of training samples, but it can also be an arbitrary numeric parameter such as `n_estimators` in a random forest.

As illustrated in the figure below, only a subset of candidates ‘survive’ until the last iteration. These are the candidates that have consistently ranked among the top-scoring candidates across all iterations. Each iteration is allocated an increasing amount of resources per candidate, here the number of samples.

<div style="text-align: center;">
  <img
    src="https://scikit-learn.org/stable/_images/sphx_glr_plot_successive_halving_iterations_001.png"
    alt="Flux de travail de recherche en grille"
    style="max-width: 50%; height: auto;">
</div>

We here briefly describe the main parameters, but each parameter and their interactions are described in more details in the sections below. The `factor` (> 1) parameter controls the rate at which the resources grow, and the rate at which the number of candidates decreases. In each iteration, the number of resources per candidate is multiplied by `factor` and the number of candidates is divided by the same factor. Along with `resource` and `min_resources`, factor is the most important parameter to control the search in our implementation, though a value of 3 usually works well. `factor` effectively controls the number of iterations in [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) and the number of candidates (by default) and iterations in [**`HalvingRandomSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html). `aggressive_elimination=True` can also be used if the number of available resources is small. More control is available through tuning the `min_resources` parameter.

These estimators are still experimental: their predictions and their API might change without any deprecation cycle. To use them, you need to explicitly import `enable_halving_search_cv`:

```python
# explicitly require this experimental feature
from sklearn.experimental import enable_halving_search_cv  # noqa
# now you can import normally from model_selection
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import HalvingRandomSearchCV
```

### Examples

#### Comparison between grid search and successive halving
#### Successive Halving Iterations

## <a id='searching-for-optimal-parameters-with-successive-halving'></a> 3.2.3. Recherche des paramètres optimaux avec le "Successive Halving"

Scikit-learn propose également les estimateurs [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) et [**`HalvingRandomSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html) qui peuvent être utilisés pour rechercher dans un espace de paramètres en utilisant le "Successive Halving" [1] [2]. Le "Successive Halving" (SH) est comme un tournoi entre les combinaisons de paramètres candidates. Le SH est un processus de sélection itératif où toutes les combinaisons candidates (les combinaisons de paramètres) sont évaluées avec une petite quantité de ressources lors de la première itération. Seules certaines de ces combinaisons candidates sont sélectionnées pour la prochaine itération, qui bénéficiera de plus de ressources. Pour l'optimisation des paramètres, la ressource est généralement le nombre d'échantillons d'entraînement, mais elle peut également être un paramètre numérique arbitraire tel que `n_estimators` dans une forêt aléatoire.

Comme illustré dans la figure ci-dessous, seuls certains candidats "survivent" jusqu'à la dernière itération. Ce sont les candidats qui se sont constamment classés parmi les meilleurs candidats en termes de score lors de toutes les itérations. À chaque itération, une quantité croissante de ressources par candidat est allouée, ici le nombre d'échantillons.

<div style="text-align: center;">
  <img
    src="https://scikit-learn.org/stable/_images/sphx_glr_plot_successive_halving_iterations_001.png"
    alt="Flux de travail de recherche en grille"
    style="max-width: 50%; height: auto;">
</div>

Nous décrivons brièvement ici les principaux paramètres, mais chaque paramètre et leurs interactions sont décrits plus en détail dans les sections suivantes. Le paramètre `factor` (> 1) contrôle le taux de croissance des ressources et le taux de diminution du nombre de candidats. À chaque itération, le nombre de ressources par candidat est multiplié par `factor` et le nombre de candidats est divisé par le même facteur. Avec `resource` et `min_resources`, le paramètre `factor` est le paramètre le plus important pour contrôler la recherche dans notre implémentation, bien qu'une valeur de 3 fonctionne généralement bien. `factor` contrôle efficacement le nombre d'itérations dans [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) et le nombre de candidats (par défaut) et d'itérations dans [**`HalvingRandomSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html). `aggressive_elimination=True` peut également être utilisé si le nombre de ressources disponibles est faible. Un contrôle plus précis est possible en ajustant le paramètre `min_resources`.

Ces estimateurs sont encore **expérimentaux** : leurs prédictions et leur API peuvent changer sans aucun cycle de dépréciation. Pour les utiliser, vous devez importer explicitement `enable_halving_search_cv` :

In [2]:
# explicitly require this experimental feature
from sklearn.experimental import enable_halving_search_cv  # noqa
# now you can import normally from model_selection
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import HalvingRandomSearchCV

### Exemples

#### [**Comparaison entre la recherche en grille et le "Successive Halving"**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_successive_halving_heatmap.ipynb)<br/>([_Comparison between grid search and successive halving_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_successive_halving_heatmap.html))

#### [**Itérations du "Successive Halving"**](https://nbviewer.org/github/Franck-PepperLabs/pepper_data-science_practising/blob/main/Sklearn/examples/3_model_selection/plot_successive_halving_iterations.ipynb)<br/>([_Successive Halving Iterations_](https://scikit-learn.org/stable/auto_examples/model_selection/plot_successive_halving_iterations.html))

### <a id='choosing-min-resources-and-the-number-of-candidates'></a> 3.2.3.1. Choix de `min_resources` et du nombre de candidats

Outre le paramètre `factor`, les deux principaux paramètres qui influencent le comportement d'une recherche "successive halving" sont le paramètre `min_resources` et le nombre de candidats (ou combinaisons de paramètres) évalués. `min_resources` correspond à la quantité de ressources allouées à chaque candidat lors de la première itération. Le nombre de candidats est spécifié directement dans [**`HalvingRandomSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html) et est déterminé à partir du paramètre `param_grid` de [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html).

Considérons un cas où la ressource est le nombre d'échantillons et où nous avons 1000 échantillons. En théorie, avec `min_resources=10` et `factor=2`, nous pouvons effectuer **au maximum** 7 itérations avec le nombre d'échantillons suivant : `[10, 20, 40, 80, 160, 320, 640]`.

Cependant, en fonction du nombre de candidats, nous pourrions exécuter moins de 7 itérations : si nous commençons avec un nombre **réduit** de candidats, la dernière itération pourrait utiliser moins de 640 échantillons, ce qui signifie que toutes les ressources disponibles (échantillons) ne sont pas utilisées. Par exemple, si nous commençons avec 5 candidats, nous n'avons besoin que de 2 itérations : 5 candidats pour la première itération, puis `5 // 2 = 2` candidats à la deuxième itération, après quoi nous saurons quel candidat est le meilleur (donc nous n'avons pas besoin d'une troisième itération). Nous n'utiliserions que 20 échantillons au maximum, ce qui serait une perte car nous avons 1000 échantillons à disposition. D'autre part, si nous commençons avec un nombre **élevé** de candidats, nous pourrions nous retrouver avec beaucoup de candidats lors de la dernière itération, ce qui peut ne pas toujours être idéal : cela signifie que de nombreux candidats fonctionneront avec toutes les ressources, réduisant ainsi essentiellement la procédure à une recherche standard.

Dans le cas de [**`HalvingRandomSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html), le nombre de candidats est défini par défaut de manière à ce que la dernière itération utilise autant de ressources disponibles que possible. Pour [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html), le nombre de candidats est déterminé par le paramètre `param_grid`. Le fait de modifier la valeur de `min_resources` aura un impact sur le nombre d'itérations possibles et, par conséquent, aura également un effet sur le nombre idéal de candidats.

Un autre facteur à prendre en compte lors du choix de `min_resources` est de savoir s'il est facile ou non de distinguer entre les bons et les mauvais candidats avec une petite quantité de ressources. Par exemple, si vous avez besoin de beaucoup d'échantillons pour distinguer entre de bons et de mauvais paramètres, il est recommandé d'utiliser une valeur élevée pour `min_resources`. En revanche, si la distinction est claire même avec une petite quantité d'échantillons, une petite valeur de `min_resources` peut être préférable car cela accélérerait le calcul.

Remarquez dans l'exemple ci-dessus que la dernière itération n'utilise pas la quantité maximale de ressources disponibles : 1000 échantillons sont disponibles, mais seuls 640 sont utilisés au maximum. Par défaut, à la fois [**`HalvingRandomSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html) et [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) essaient d'utiliser autant de ressources que possible lors de la dernière itération, avec la contrainte que cette quantité de ressources doit être un multiple à la fois de `min_resources` et `factor` (cette contrainte sera expliquée dans la prochaine section). [**`HalvingRandomSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html) atteint cet objectif en échantillonnant la bonne quantité de candidats, tandis que [**`HalvingGridSearchCV`**](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html) y parvient en fixant correctement `min_resources`. Pour plus de détails, veuillez consulter [**Épuisement des ressources disponibles** (3.2.3.4)](https://scikit-learn.org/stable/modules/grid_search.html#exhausting-the-resources).